<a href="https://colab.research.google.com/github/Qu1nnD/CS290/blob/main/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from collections import defaultdict
from sklearn.tree import plot_tree
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [80]:
shrooms = pd.read_csv("https://raw.githubusercontent.com/Qu1nnD/CS290/main/data/mushroom_cleaned.csv")
shrooms.describe()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
count,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000
mean,567.257204,4.000315,2.142056,7.329509,0.759110,1051.081299,8.418062,0.952163,0.549181
std,359.883763,2.160505,2.228821,3.200266,0.650969,782.056076,3.262078,0.305594,0.497580
min,0.000000,0.000000,0.000000,0.000000,0.000426,0.000000,0.000000,0.027372,0.000000
25%,289.000000,2.000000,0.000000,5.000000,0.270997,421.000000,6.000000,0.888450,0.000000
50%,525.000000,5.000000,1.000000,8.000000,0.593295,923.000000,11.000000,0.943195,1.000000
75%,781.000000,6.000000,4.000000,10.000000,1.054858,1523.000000,11.000000,0.943195,1.000000
max,1891.000000,6.000000,6.000000,11.000000,3.835320,3569.000000,12.000000,1.804273,1.000000


In [81]:
shrooms.head()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
0,1372,2,2,10,3.807467,1545,11,1.804273,1
1,1461,2,2,10,3.807467,1557,11,1.804273,1
2,1371,2,2,10,3.612496,1566,11,1.804273,1
3,1261,6,2,10,3.787572,1566,11,1.804273,1
4,1305,6,2,10,3.711971,1464,11,0.943195,1


In [82]:
shrooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54035 entries, 0 to 54034
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cap-diameter     54035 non-null  int64  
 1   cap-shape        54035 non-null  int64  
 2   gill-attachment  54035 non-null  int64  
 3   gill-color       54035 non-null  int64  
 4   stem-height      54035 non-null  float64
 5   stem-width       54035 non-null  int64  
 6   stem-color       54035 non-null  int64  
 7   season           54035 non-null  float64
 8   class            54035 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 3.7 MB


In [83]:
features = shrooms.drop(columns=['class'])
target = shrooms['class']
poison_instance = {'cap-diameter': 1372, 'cap-shape': 2, 'gill-attachment': 2, 'gill-color': 10, 'stem-height': 3.807, 'stem-width':1545 , 'stem-color': 11}
non_poison_instance = {'cap-diameter': 914, 'cap-shape': 2, 'gill-attachment': 0, 'gill-color': 10, 'stem-height': 1.47, 'stem-width':1418 , 'stem-color': 11}
new_instance1 = {'cap-diameter': 1000, 'cap-shape': 2, 'gill-attachment': 1, 'gill-color': 10, 'stem-height': 2.5, 'stem-width':1200 , 'stem-color': 11}
new_instance2 = {'cap-diameter': 950, 'cap-shape': 2, 'gill-attachment': 0, 'gill-color': 10, 'stem-height': 2.35, 'stem-width':1400 , 'stem-color': 7}

In [84]:
def naive_bayes_classifier(features, target, new_instance):
    classes = target.unique()
    priors = {}
    likelihoods = defaultdict(dict)
    for cls in classes:
        priors[cls] = len(target[target == cls]) / len(target)
        features_cls = features[target == cls]

        for col in features.columns:
            if features[col].nunique() < 20:
                feature_probs = features_cls[col].value_counts(normalize=True)
                likelihoods[col][cls] = feature_probs
            else:
                mean = features_cls[col].mean()
                std = features_cls[col].std()
                likelihoods[col][cls] = (mean, std)

    probs_dict = {}
    for cls in classes:
        probs = np.log(priors[cls])
        for feature, value in new_instance.items():
            if feature in likelihoods:
                if features[feature].nunique() < 20:
                    likelihood = likelihoods[feature][cls].get(value, 1e-6)
                else:
                    mean, std = likelihoods[feature][cls]
                    likelihood = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-((value - mean) ** 2) / (2 * std ** 2))
                probs += np.log(likelihood)
        probs_dict[cls] = probs
    return max(probs_dict, key=probs_dict.get)

In [85]:
predicted_class1=naive_bayes_classifier(features,target,poison_instance)
predicted_class1

1

In [86]:
predicted_class2=naive_bayes_classifier(features,target,non_poison_instance)
predicted_class2

0

In [87]:
predicted_class3=naive_bayes_classifier(features,target,new_instance1)
predicted_class3

1

In [88]:
predicted_class4=naive_bayes_classifier(features,target,new_instance2)
predicted_class4

1

In [89]:
train_set, test_set = train_test_split(shrooms, test_size=0.2)
train_set = train_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color', 'stem-height', 'stem-width', 'stem-color', 'class']]
test_set = test_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color', 'class']]
X_train = train_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color']]
y_train = train_set["class"]
X_test = test_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color']]
y_test = test_set["class"]
cat_attributes=[]
num_attributes=[]
for col in X_train.columns:
  if len(X_train[col].unique()) < 20:
    cat_attributes.append(col)
  else:
    num_attributes.append(col)
trf = [ ('num', StandardScaler(), num_attributes),
       ('cat', OneHotEncoder( handle_unknown='ignore'), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )
pipeline = Pipeline( steps = [('pre', col_transform),
 ('clf', DecisionTreeClassifier(max_depth=3))])

In [90]:
cv_scores=cross_val_score( pipeline, X_train, y_train, cv=5, scoring='accuracy')
cv_scores

array([0.66250289, 0.6820495 , 0.66897988, 0.6816657 , 0.67218045])

In [91]:
base_DT_model = pipeline.fit( X_train, y_train )
base_DT_model

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['cap-diameter',
                                                   'stem-height',
                                                   'stem-width']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cap-shape',
                                                   'gill-attachment',
                                                   'gill-color',
                                                   'stem-color'])])),
                ('clf', DecisionTreeClassifier(max_depth=3))])

In [92]:
y_pred = base_DT_model.predict(X_test)
y_pred

array([0, 1, 1, ..., 0, 1, 1])

In [93]:
print("Base Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


Base Model Performance:
Accuracy: 0.67


In [94]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print("Tuned Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.2f}")


Tuned Model Performance:
Accuracy: 0.98


In [95]:
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train)
y_pred_dt = decision_tree_model.predict(X_test)
print("Decision Tree Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.2f}")


Decision Tree Model Performance:
Accuracy: 0.98


In [96]:
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
y_pred_rf = random_forest_model.predict(X_test)
print("Random Forest Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")


Random Forest Model Performance:
Accuracy: 0.99
